# Finite-State Machines

In [3]:
from jmodel import JModel
from transitions import *
from transitions.extensions import GraphMachine

        
# transitions = [
#     { 'trigger': 'melt', 'source': 'solid', 'dest': 'liquid' },
#     { 'trigger': 'evaporate', 'source': 'liquid', 'dest': 'gas', 'conditions':'is_valid' },
#     { 'trigger': 'sublimate', 'source': 'solid', 'dest': 'gas', 'unless':'is_not_valid' },
#     { 'trigger': 'ionize', 'source': 'gas', 'dest': 'plasma', 
#       'conditions':['is_valid','is_also_valid'] }
# ]
# states=['solid', 'liquid', 'gas', 'plasma']

transitions = [
    { 'trigger': 'insert card', 'source': 'start', 'dest': '1' },
    { 'trigger': 'wrong PIN', 'source': '1', 'dest': 'start' },
    { 'trigger': 'PIN ok', 'source': '1', 'dest': '2' },
    { 'trigger': 'withdraw', 'source': '2', 'dest': '4' },
    { 'trigger': 'balance', 'source': '2', 'dest': '3' },
    { 'trigger': 'return card', 'source': '3', 'dest': 'start' },
    { 'trigger': 'choose a/c', 'source': '4', 'dest': '5' },
    { 'trigger': 'amount', 'source': '5', 'dest': 'ok?' },
    { 'trigger': 'no', 'source': 'ok?', 'dest': '5' },
    { 'trigger': 'yes', 'source': 'ok?', 'dest': '6' },
    { 'trigger': 'return card', 'source': '6', 'dest': '£' },
    { 'trigger': 'take money', 'source': '£', 'dest': 'start' }
]
states = ['start', '1', '2', '3', '4', '5', 'ok?', '6', '£']

model = JModel()
machine = GraphMachine(model=model, 
                       states=states, 
                       transitions=transitions,
                       initial='start',
#                        show_auto_transitions=True, # default value is False
                       title="ATM machine",
                       show_conditions=True)
model.show_graph()

Exception: AGraph diagram requires pygraphviz

FSM allow you to describe the states and transitions of a system, such as an ATM above.

In [2]:
transitions = [
#     { 'trigger': 'insert card', 'source': 'start', 'dest': '1' },
    {'trigger': '0', 'source': 's0', 'dest': 's0'},
    {'trigger': '0', 'source': 's1', 'dest': 's3'},
    {'trigger': '0', 'source': 's2', 'dest': 's2'},
    {'trigger': '0', 'source': 's3', 'dest': 's1'},
    
    {'trigger': '1', 'source': 's0', 'dest': 's1'},
    {'trigger': '1', 'source': 's1', 'dest': 's0'},
    {'trigger': '1', 'source': 's2', 'dest': 's3'},
    {'trigger': '1', 'source': 's3', 'dest': 's2'},
    
    {'trigger': '2', 'source': 's0', 'dest': 's2'},
    {'trigger': '2', 'source': 's1', 'dest': 's1'},
    {'trigger': '2', 'source': 's2', 'dest': 's0'},
    {'trigger': '2', 'source': 's3', 'dest': 's3'},
]
states = ['s'+str(i) for i in range(0, 4)]

model = JModel()
machine = GraphMachine(model=model, 
                       states=states, 
                       transitions=transitions,
                       initial='s0',
#                        show_auto_transitions=True, # default value is False
                       title="",
                       show_conditions=True)
model.show_graph()

Exception: AGraph diagram requires pygraphviz

The machine has 4 states
$$
Q = \{s0, s1, s2, s3\}
$$

And tree symbols that form the alphabet (labels the transitions)
$$
\Sigma = \{0, 1, 2\}
$$

The FSM is a special case since their is exactly one transition form a state for each label. This means the next state is can be determined by the previous state and the transition. This special case is called a deterministic finitie-state automaton. 

A DFA will have a next state function (since the next state can be determined by the previous state and the transition). We can represent this function as a table:

|    | 0  | 1  | 2  |
| --- | --- | --- | --- |
| __s0__ | s0 | s1 | s2 |
| __s1__ | s3 | s0 | s1 |
| __s2__ | s2 | s3 | s0 |
| __s3__ | s1 | s2 | s3 |

Alternativly we can represent the transitions by a relation, what symbol takes us from state __q__ to state __r__:

| q\\r | s0 | s1 | s2 | s3 |
| --- | --- | --- | --- | --- |
| __s0__ | 0 | 1 | 2 | - |
| __s1__ | 1 | 2 | - | 0 |
| __s2__ | 2 | - | 0 | 1 |
| __s3__ | - | 0 | 1 | 2 |

To figure out what this machine does, lets look at the state we finish in with diffrent inputs

| Decimal input | Tenary | Path |
| :--- | :--- | :--- |
| 0 | 0   | S0 |
| 1 | 1   | S1 |
| 2 | 2   | S2 |
| 3 | 10  | S1 S3 |
| 4 | 11  | S1 S0 |
| 5 | 12  | S1 S1 |
| 6 | 20  | S2 S2 |
| 7 | 21  | S2 S3 |
| 8 | 22  | S2 S0 |
| 9 | 100 | S1 S3 S1 |

We can see the the machine counts in tenary mod 4 since the last state is looping every 4th input. Using this we can derive the next state function:
$$
F(q, s) = (3 \times q + s) \bmod 4
$$
where $q$ is the current state, and $s$ is the transition.